## Welcome to the HWComponents Tutorial!

This tutorial will guide you through the process of using pre-existing models in the
HWComponents library.

To start, let's import the necessary modules.

In [1]:
import logging
logging.getLogger().setLevel(logging.WARNING)
import hwcomponents as hwc

### Listing available models

``hwcomponents`` supports many different component models. We can list available
component models with the ``get_models`` function. This function returns a list of
``ComponentModel`` subclasses.

You may also use the `hwcomponents --list` command from the shell.

In [2]:
# < DOC_INCLUDE_MARKER > listing_available_models
import hwcomponents as hwc

# Find and list all installed component models
print(f'Listing component models')
all_installed_models = hwc.get_models()
print(f'Number of available component models: {len(all_installed_models)}')
for model in all_installed_models[:5]:
    print(f'{model.__name__} supports {model.component_name}')

Listing component models
Number of available component models: 75
ADC supports ['ArrayADC', 'NeuroSimArrayADC']
Adder supports ['Adder', 'NeuroSimAdder', 'IntAdder']
AdderTree supports ['AdderTree', 'NeuroSimAdderTree', 'IntAdderTree']
AladdinAdder supports ['Adder', 'AladdinAdder', 'IntAdder']
AladdinComparator supports ['Comparator', 'AladdinComparator']


If we know what type of component we would like to model, we can use the
`name_must_include` argument to `get_models` to find all models that match a given class
name.

We'll use the `hwcomponents_cacti` package to model an SRAM. Once we've found the model,
we can use the `help` function to see its documentation and supported actions.

In [3]:
# < DOC_INCLUDE_MARKER > finding_components_2

# Find an SRAM model
print(f'\nListing SRAM models')
sram_models = hwc.get_models(name_must_include='sram')
print(f'Number of SRAM models: {len(sram_models)}')
for model in sram_models[:5]:
    print(f'{model.__name__} supports {model.component_name}')

# Grab the CACI SRAM model & use the "help" function to see its documentation
models = [s for s in sram_models if "hwcomponents_cacti" in s.__module__]
assert len(models) == 1, \
    f"Excected 1 CACTI SRAM model, got {len(models)}. Is hwcomponents_cacti installed?"
sram = models[0]
help(sram)


Listing SRAM models
Number of SRAM models: 2
SRAM supports ['SRAM', 'sram']
SmartBufferSRAM supports ['smart_buffer_sram', 'smartbuffer_sram', 'smartbuffersram']
Help on class SRAM in module hwcomponents_cacti.hwcomponents_cacti:

class SRAM(_Memory)
 |  SRAM(tech_node: float, width: int, depth: int, n_rw_ports: int = 1, n_banks=1)
 |
 |  SRAM model using CACTI.
 |
 |  Parameters
 |  ----------
 |      tech_node: The technology node of the SRAM in meters.
 |      width: The width of the read and write ports in bits. This is the number of bits
 |          that are accssed by any one read/write. Total size = width * depth.
 |      depth: The number of entries in the SRAM, each with `width` bits. Total size =
 |          width * depth.
 |      n_rw_ports: The number of read/write ports. Bandwidth will increase with more
 |          ports.
 |      n_banks: The number of banks. Bandwidth will increase with more banks.
 |
 |  Attributes
 |  ----------
 |      component_name: set to "sram"
 

Once we know the model we'd like to use, we can import the model directly
and instantiate components.

In [4]:
# < DOC_INCLUDE_MARKER > importing_models

# Now that we have a component model, we can use it to create components.
from hwcomponents_cacti.hwcomponents_cacti import SRAM

# Create an SRAM component using the CACTI model
sram = SRAM(
    tech_node=16e-9,
    width=1024,
    depth=256,
)
print(f'SRAM read energy: {sram.read(bits_per_action=8)} J')
print(f'SRAM write energy: {sram.write(bits_per_action=8)} J')
print(f'SRAM area: {sram.area} m^2')
print(f'SRAM leak power: {sram.leak_power} W')

SRAM read energy: (1.8941884269838074e-13, 0.0) J
SRAM write energy: (2.7776189078152096e-13, 0.0) J
SRAM area: 3.4692646522688153e-06 m^2
SRAM leak power: 3.198538181818182e-08 W


If you're unsure of which component model you'd like to use, there are other ways to
invoke a model. There are three ways to find a component model:

1. Import the model from a module and use it directly.
2. Ask hwcomponents to select the best model for a given component. hwcomponents will
   select the best model for a given component name and attributes, and raise an error
   if no model can be instantiated with the given attributes.
3. Ask for specific properties from hwcomponents. This is similar to the second method,
   but you can ask for the energy, area, or leakage power of a component directly.

In [5]:
# < DOC_INCLUDE_MARKER > ways_to_find_components

# Method 1: Import the model from a module and use it directly.
from hwcomponents_cacti import SRAM
sram = SRAM(
    tech_node=40e-9, # 40nm
    width=64,
    depth=1024,
)
read_energy, read_latency = sram.read()
print(f"SRAM read energy is {read_energy:.2e}J and read latency is {read_latency:.2e}s")
print(f"SRAM area is {sram.area:.2e}m^2. Leak power is {sram.leak_power:.2e}W")


# Method 2: Ask hwcomponents to select the best model for a given component.
model = hwc.get_model(
    component_name="SRAM", # These are NOT case sensitive.
    component_attributes={
        "tech_node": 40e-9, # 40nm
        "width": 64,
        "depth": 1024
    },
    required_actions=["read"]
)
read_energy, read_latency = model.read()
print(f'Read energy is {read_energy:.2e}J and read latency is {read_latency:.2e}s')
print(f'Area is {model.area:.2e}m^2. Leak power is {model.leak_power:.2e}W')

# Method 3: Ask for specific properties from hwcomponents
attributes = {
    "tech_node": 40e-9, # 40nm
    "width": 64,
    "depth": 1024
}

read_energy = hwc.get_energy(
    component_name="SRAM",
    component_attributes=attributes,
    action_name="read",
    action_arguments={}
)
read_latency = hwc.get_latency(
    component_name="SRAM",
    component_attributes=attributes,
    action_name="read",
    action_arguments={}
)
area = hwc.get_area(
    component_name="SRAM",
    component_attributes=attributes,
)
leak_power = hwc.get_leak_power(
    component_name="SRAM",
    component_attributes=attributes,
)
print(f'Read energy is {read_energy:.2e}J and read latency is {read_latency:.2e}s')
print(f'Area is {area:.2e}m^2. Leak power is {leak_power:.2e}W')

SRAM read energy is 5.10e-12J and read latency is 0.00e+00s
SRAM area is 3.31e-07m^2. Leak power is 2.13e-08W
Return estimation object: False
Read energy is 5.10e-12J and read latency is 0.00e+00s
Area is 3.31e-07m^2. Leak power is 2.13e-08W
Return estimation object: False
Return estimation object: False
Return estimation object: False
Return estimation object: False


TypeError: unsupported format string passed to FloatEstimation.__format__